In [8]:
import os
from pathlib import Path
import matlab.engine
import numpy as np
import shutil
import tempfile
import json

# Base directory (adjust to your setup)
base_dir = Path("./")

 # --- Start MATLAB Engine ---
eng = matlab.engine.start_matlab()

 # --- Add SCOPE path to MATLAB's search path ---
scope_path = './SCOPE'  # e.g., '/home/user/Documents/SCOPE'
eng.addpath(scope_path, nargout=0)

# MODTRAN paths
modtran_dir = base_dir / "MODTRAN5"
modtran_exe = modtran_dir / "bin" / "Mod5_mac.exe" 
modtran_tp5_template = modtran_dir / "HyPlant-FLUO_Modtran5_base_v1.tp5"

# SCOPE paths
scope_dir = base_dir / "SCOPE"
scope_main = scope_dir / "SCOPE.m"
scope_wrapper = scope_dir / "run_scope_wrapper.m"

# Output directory
output_dir = base_dir / "synthetic_dataset"
output_dir.mkdir(exist_ok=True)

print(f"Checking path: {scope_dir.resolve()}")
print(f"Directory exists: {scope_dir.exists()}")
print(f"Directory contents: {list(scope_dir.glob('*')) if scope_dir.exists() else 'N/A'}")


Checking path: /Users/mirkomorello/Documents/Università/MSc_Sensors_Imaging/Final_Project/SCOPE
Directory exists: True
Directory contents: [PosixPath('SCOPE/.DS_Store'), PosixPath('SCOPE/run_scope_wrapper_json.m'), PosixPath('SCOPE/input'), PosixPath('SCOPE/output'), PosixPath('SCOPE/docs'), PosixPath('SCOPE/soltir_tp7.m'), PosixPath('SCOPE/SCOPE.m'), PosixPath('SCOPE/README.md'), PosixPath('SCOPE/.gitignore'), PosixPath('SCOPE/set_parameter_filenames.csv'), PosixPath('SCOPE/bug_reports.txt'), PosixPath('SCOPE/GNU_General_Public_Licence.txt'), PosixPath('SCOPE/.gitattributes'), PosixPath('SCOPE/run_scope_wrapper.m'), PosixPath('SCOPE/.git'), PosixPath('SCOPE/.readthedocs.yaml'), PosixPath('SCOPE/SCOPE.exe'), PosixPath('SCOPE/src')]


In [ ]:
import csv
import os
import shutil
import json
import matlab.engine
from pathlib import Path
import numpy as np

class SCOPEWrapper:
    def __init__(self):
        self.base_dir = Path.cwd()  # More robust; use current working directory
        self.scope_dir = self.base_dir / "SCOPE"
        self.original_input = self.scope_dir / "input" / "input_data.csv"
        self.default_params = self._get_default_parameters()
        self.eng = None  # Initialize engine to None

    def _get_default_parameters(self):
        """Comprehensive default parameters for SCOPE (as before)."""
        return {
            # PROSPECT
            'Cab': 40.0, 'Cca': 10.0, 'Cdm': 0.012, 'Cw': 0.009,
            'Cs': 0.0, 'Cant': 1.0, 'Cp': 0.0, 'Cbc': 0.0,
            'N': 1.5, 'rho_thermal': 0.01, 'tau_thermal': 0.01,
            
            # Leaf Biochemical
            'Vcmax25': 60.0, 'BallBerrySlope': 8.0, 'BallBerry0': 0.01,
            'Type': 0.0, 'kV': 0.64, 'Rdparam': 0.015, 'Kn0': 2.48,
            'Knalpha': 2.83, 'Knbeta': 0.114,
            
            # Magnani
            'Tyear': 15.0, 'beta': 0.51, 'kNPQs': 0.0, 
            'qLs': 1.0, 'stressfactor': 1.0,
            
            # Fluorescence
            'fqe': 0.01,
            
            # Soil
            'spectrum': 1.0, 'rss': 500.0, 'rs_thermal': 0.06,
            'cs': 1180.0, 'rhos': 1800.0, 'lambdas': 1.55,
            'SMC': 25.0, 'BSMBrightness': 0.5, 'BSMlat': 25.0,
            'BSMlon': 45.0,
            
            # Canopy
            'LAI': 3.0, 'hc': 2.0, 'LIDFa': -0.35, 'LIDFb': -0.15,
            'leafwidth': 0.1, 'Cv': 1.0, 'crowndiameter': 1.0,
            
            # Meteo
            'z': 5.0, 'Rin': 800.0, 'Ta': 20.0, 'Rli': 300.0,
            'p': 970.0, 'ea': 15.0, 'u': 2.0, 'Ca': 410.0, 'Oa': 209.0,
            
            # Aerodynamic
            'zo': 0.25, 'd': 1.34, 'Cd': 0.3, 'rb': 10.0,
            'CR': 0.35, 'CD1': 20.6, 'Psicor': 0.2,
            'CSSOIL': 0.01, 'rbs': 10.0, 'rwc': 0.0,
            
            # Timeseries
            'startDOY': 20060618.0, 'endDOY': 20300101.0,
            'LAT': 51.55, 'LON': 5.55, 'timezn': 1.0,
            
            # Angles
            'tts': 35.0, 'tto': 0.0, 'psi': 0.0
        }

    def __enter__(self):
        """Start MATLAB engine and add SCOPE path."""
        self.eng = matlab.engine.start_matlab('-nodisplay')  # Add -nodisplay for no GUI
        self.eng.addpath(self.eng.genpath(str(self.scope_dir)), nargout=0)  # Use genpath
        self.eng.cd(str(self.scope_dir), nargout=0)  # Change to SCOPE directory
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        """Stop MATLAB engine and restore input file."""
        if self.eng:
            self.eng.quit()

         # Restore original file (important for repeated runs)
        if hasattr(self, 'temp_backup') and self.temp_backup.exists():
            try:
                if self.original_input.exists():
                    os.remove(str(self.original_input))
                shutil.move(str(self.temp_backup), str(self.original_input))
            except Exception as e:
                print(f"Error restoring original input file: {e}")



    def _prepare_input(self, csv_content):
        """Write input directly to SCOPE's input directory"""
        input_dir = self.scope_dir / "input"
        input_dir.mkdir(parents=True, exist_ok=True)  # Ensure the directory exists
        temp_file = input_dir / "input_data.csv"

        # Backup original file if it exists
        if self.original_input.exists():
            self.temp_backup = self.original_input.with_name("input_data.backup.csv")
            if self.temp_backup.exists():
                os.remove(str(self.temp_backup)) # Delete previous backup to avoid errors.
            shutil.move(str(temp_file), str(self.temp_backup))

        # Write new content
        with open(temp_file, 'w') as f:
            f.write(csv_content)
        return temp_file
            

    def _generate_csv_content(self, params):
        """Generate CSV content with exact SCOPE formatting (same as before)."""
        sections = [
            ('PROSPECT', [
                'Cab', 'Cca', 'Cdm', 'Cw', 'Cs', 'Cant', 
                'Cp', 'Cbc', 'N', 'rho_thermal', 'tau_thermal'
            ]),
            ('Leaf_Biochemical', [
                'Vcmax25', 'BallBerrySlope', 'BallBerry0', 'Type',
                'kV', 'Rdparam', 'Kn0', 'Knalpha', 'Knbeta'
            ]),
            ('Leaf_Biochemical_magnani', [
                'Tyear', 'beta', 'kNPQs', 'qLs', 'stressfactor'
            ]),
            ('Fluorescence', ['fqe']),
            ('Soil', [
                'spectrum', 'rss', 'rs_thermal', 'cs', 'rhos',
                'lambdas', 'SMC', 'BSMBrightness', 'BSMlat', 'BSMlon'
            ]),
            ('Canopy', [
                'LAI', 'hc', 'LIDFa', 'LIDFb', 'leafwidth',
                'Cv', 'crowndiameter'
            ]),
            ('Meteo', [
                'z', 'Rin', 'Ta', 'Rli', 'p', 'ea', 'u', 'Ca', 'Oa'
            ]),
            ('Aerodynamic', [
                'zo', 'd', 'Cd', 'rb', 'CR', 'CD1', 'Psicor',
                'CSSOIL', 'rbs', 'rwc'
            ]),
            ('timeseries', [
                'startDOY', 'endDOY', 'LAT', 'LON', 'timezn'
            ]),
            ('Angles', ['tts', 'tto', 'psi'])
        ]
        content = []
        for section, parameters in sections:
            content.append(f"{section},")
            for param in parameters:
                value = params.get(param, self.default_params[param])  # Use get with default
                content.append(f"{param},{float(value)}")
            content.append(",")
            
        return "\r\n".join(content) + "\r\n"



    def _find_latest_output(self):
        """Find the newest output directory (same as before)."""
        output_parent = self.scope_dir / "output"
        runs = sorted(output_parent.glob("example_run_*"), key=os.path.getmtime, reverse=True)
        if not runs:
            raise FileNotFoundError("No SCOPE output directories found.")
        return runs[0]

    def _read_data(self, filepath):
        """Reads data from a file, handling both CSV and TXT."""
        if filepath.suffix.lower() == '.csv':
            try:
                # Skip header row, handle comma as decimal
                return np.loadtxt(filepath, delimiter=',', skiprows=1, dtype=float).tolist()
            except Exception as e:
                print(f"Error reading CSV file {filepath}: {e}")
                return None  # Or [] or {} depending on context
        elif filepath.suffix.lower() == '.txt':
            try:
                wavelengths = []
                with open(filepath, 'r') as f:
                    for line in f:
                        # Split each line by spaces and convert to floats
                        values = [float(value) for value in line.split()]
                        wavelengths.extend(values)
                return wavelengths
            except Exception as e:
                print(f"Error reading TXT file {filepath}: {e}")
                return None
        else:
            print(f"Unsupported file type: {filepath.suffix}")
            return None
    
    
    def _read_parameters(self, filepath):
        """Reads parameters from a CSV file, handling section headers."""
        params = {}
        with open(filepath, 'r') as f:
            reader = csv.reader(f)
            current_section = None
            for row in reader:
                if not row:  # Skip empty lines
                    continue
                if row[0].endswith(','):  # Section header
                    current_section = row[0].strip(',')
                elif len(row) >= 2 and current_section:
                    param_name = row[0].strip()
                    param_value = row[1].strip()
                    # Convert to float if possible, otherwise keep as string
                    try:
                        param_value = float(param_value)
                    except ValueError:
                        pass  # Keep as string if conversion fails
                    params[f"{current_section}.{param_name}"] = param_value
        return params


    def generate_results_json(self, output_dir: Path) -> Path:
        """Generate a comprehensive results.json from all SCOPE outputs."""
        results = {}
        # Process the Parameters subdirectory
        params_dir = output_dir / "Parameters"
        if params_dir.exists() and params_dir.is_dir():
            for param_file in params_dir.glob("*.csv"):
                if param_file.name.startswith("filenames"):
                    # Special handling for filenames
                    filenames = {}
                    with open(param_file, 'r') as f:
                        reader = csv.reader(f)
                        for row in reader:
                            if len(row) >= 2:
                                filenames[row[0].strip()] = row[1].strip()
                    results['filenames'] = filenames
                elif param_file.name.startswith("input_data"):
                    # Handle input_data separately
                    results['input_data'] = self._read_parameters(param_file)
                elif param_file.name.startswith("setoptions"):
                    # Handle setoptions separately
                    results['setoptions'] = self._read_parameters(param_file)

        # Read data files (CSV and TXT) in the main output directory
        for file in output_dir.iterdir():
            if file.is_file():  # Only process files
                data = self._read_data(file)
                if data is not None:
                    results[file.stem] = data
        
        
        json_path = output_dir / "results.json"
        with open(json_path, 'w') as f:
            json.dump(results, f, indent=2)

        return json_path

    def _write_scope_csv(self, params):
        """Write parameters directly to SCOPE's input_data.csv"""
        input_path = self.scope_dir / "input" / "input_data.csv"
        
        # Generate CSV content using your existing method
        csv_content = self._generate_csv_content(params)  
        
        # Write directly to SCOPE's expected input location
        with open(input_path, 'w', newline='') as f:
            f.write(csv_content)

    def run(self, params):
      """Run SCOPE with the given parameters and return parsed output."""
      try:
          # Write parameters to input file.
          self._write_scope_csv(params)

          # Run SCOPE (ensure the MATLAB script is in the MATLAB path).
          # The `nargout=0` is crucial to prevent hanging.
          self.eng.run_scope_wrapper_json(nargout=0)

          # Find the output directory.
          output_dir = self._find_latest_output()

          # Process the output and return it.
          json_path = self.generate_results_json(output_dir)
          return json_path


      except matlab.engine.MatlabExecutionError as e:
          print(f"MATLAB Error: {str(e)}")
          raise  # Re-raise to halt execution
      except FileNotFoundError as e:
          print(f"File Not Found Error: {str(e)}")
          raise
      except Exception as e:
          print(f"An unexpected error occurred: {str(e)}")
          raise

In [96]:
import pandas as pd

if __name__ == "__main__":
    #Diagnostic output
    print("\n=== Path Diagnostics ===")
    print(f"Current working directory: {Path.cwd()}")
    print(f"Checking for 'SCOPE' in: {Path.cwd()}")

    try:
        with SCOPEWrapper() as scope:
            print("\n=== Running Test Simulation ===")
            # Example parameters (can be adjusted)
            test_params = {
                'Cab': 45.0,
                'LAI': 3.2,
                'hc': 1.5,
                'tts': 35.0,
                'Rin': 800.0
            }
            results_path = scope.run(test_params)
            print("Success!")
            # You can now load and inspect the results.json here if needed
            with open(results_path, 'r') as f:
                results = json.load(f)
            print("\n=== Results ===")
            print(json.dumps(results, indent=2))
    except Exception as e:
        print(f"Error during SCOPE execution: {e}")


=== Path Diagnostics ===
Current working directory: /Users/mirkomorello/Documents/Università/MSc_Sensors_Imaging/Final_Project
Checking for 'SCOPE' in: /Users/mirkomorello/Documents/Università/MSc_Sensors_Imaging/Final_Project

=== Running Test Simulation ===

 Do not quench your inspiration and your imagination; do not become the slave of your model (Vincent van Gogh). 
simulation 1 of 1 start now 
Elapsed time is 0.487827 seconds.
Success!

=== Results ===
{
  "filenames": {
    "Do_not_enter_additional_colums_of_text_in_this_sheet": "",
    "": "",
    "The_following_three_are_always_required": "",
    "Simulation_Name": "example_run_2",
    "optipar_file": "Optipar2021_ProspectPRO_CX.mat",
    "atmos_file": "FLEX-S3_std.atm",
    "soil_file": "",
    "mSCOPE_csv": "",
    "nly": "3",
    "atmos_names": "",
    "The_following_are_only_for_the_time_series_option!": "",
    "Dataset_dir": "for_verification",
    "meteo_ec_csv": "input_data_latin_hypercube_ts.csv",
    "t_file": "",

In [127]:
# This is an experiment at changing SCOPEWrapper to handle multiple runs
# It is not complete and may require further adjustments, it is meant as a starting point.

import csv
import os
import shutil
import json
import matlab.engine
from pathlib import Path
import numpy as np
import pandas as pd
import struct  # Import the 'struct' module
import re



class SCOPEWrapperMultiRun:
    def __init__(self):
        self.base_dir = Path.cwd()
        self.scope_dir = self.base_dir / "SCOPE"
        self.original_input = self.scope_dir / "input" / "input_data.csv"
        self.default_params = self._get_default_parameters()
        self.eng = None
        self.setoptions = self._get_default_setoptions() # Load default setoptions


    def _get_default_setoptions(self):
        """Loads default setoptions from setoptions.csv"""
        setoptions_path = self.scope_dir / "input" / "setoptions.csv"
        setoptions = {}
        with open(setoptions_path, 'r') as f:
            reader = csv.reader(f)
            for row in reader:
                if row and len(row) >= 2:  # Ensure row is not empty and has key-value
                    key = row[0].strip()
                    value = row[1].strip()
                    try:
                        # Convert to int if possible
                        value = int(value)
                    except ValueError:
                        pass  # Keep as string if conversion fails
                    setoptions[key] = value
        return setoptions

    def _get_default_parameters(self):
        """Comprehensive default parameters for SCOPE."""
        return {
            # PROSPECT
            'Cab': 40.0, 'Cca': 10.0, 'Cdm': 0.012, 'Cw': 0.009,
            'Cs': 0.0, 'Cant': 1.0, 'Cp': 0.0, 'Cbc': 0.0,
            'N': 1.5, 'rho_thermal': 0.01, 'tau_thermal': 0.01,

            # Leaf Biochemical
            'Vcmax25': 60.0, 'BallBerrySlope': 8.0, 'BallBerry0': 0.01,
            'Type': 0.0, 'kV': 0.64, 'Rdparam': 0.015, 'Kn0': 2.48,
            'Knalpha': 2.83, 'Knbeta': 0.114,

            # Magnani
            'Tyear': 15.0, 'beta': 0.51, 'kNPQs': 0.0,
            'qLs': 1.0, 'stressfactor': 1.0,

            # Fluorescence
            'fqe': 0.01,

            # Soil
            'spectrum': 1.0, 'rss': 500.0, 'rs_thermal': 0.06,
            'cs': 1180.0, 'rhos': 1800.0, 'lambdas': 1.55,
            'SMC': 25.0, 'BSMBrightness': 0.5, 'BSMlat': 25.0,
            'BSMlon': 45.0,

            # Canopy
            'LAI': 3.0, 'hc': 2.0, 'LIDFa': -0.35, 'LIDFb': -0.15,
            'leafwidth': 0.1, 'Cv': 1.0, 'crowndiameter': 1.0,

            # Meteo
            'z': 5.0, 'Rin': 800.0, 'Ta': 20.0, 'Rli': 300.0,
            'p': 970.0, 'ea': 15.0, 'u': 2.0, 'Ca': 410.0, 'Oa': 209.0,

            # Aerodynamic
            'zo': 0.25, 'd': 1.34, 'Cd': 0.3, 'rb': 10.0,
            'CR': 0.35, 'CD1': 20.6, 'Psicor': 0.2,
            'CSSOIL': 0.01, 'rbs': 10.0, 'rwc': 0.0,

            # Timeseries
            'startDOY': 20060618.0, 'endDOY': 20300101.0,
            'LAT': 51.55, 'LON': 5.55, 'timezn': 1.0,

            # Angles
            'tts': 35.0, 'tto': 0.0, 'psi': 0.0
        }

    def __enter__(self):
        """Start MATLAB engine and add SCOPE path."""
        self.eng = matlab.engine.start_matlab('-nodisplay')
        self.eng.addpath(self.eng.genpath(str(self.scope_dir)), nargout=0)
        self.eng.cd(str(self.scope_dir), nargout=0)
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        """Stop MATLAB engine and restore input file."""
        if self.eng:
            self.eng.quit()

        if hasattr(self, 'temp_backup') and self.temp_backup.exists():
            try:
                if self.original_input.exists():
                    os.remove(str(self.original_input))
                shutil.move(str(self.temp_backup), str(self.original_input))
            except Exception as e:
                print(f"Error restoring original input file: {e}")



    def _prepare_input(self, csv_content):
        """Write input directly to SCOPE's input directory, backing up the original."""
        input_dir = self.scope_dir / "input"
        input_dir.mkdir(parents=True, exist_ok=True)
        temp_file = input_dir / "input_data.csv"

        # Backup original file if it exists
        if self.original_input.exists():
            self.temp_backup = self.original_input.with_name("input_data.backup.csv")
            if self.temp_backup.exists():
                os.remove(str(self.temp_backup))
            shutil.move(str(self.original_input), str(self.temp_backup))

        # Write new content
        with open(temp_file, 'w') as f:
            f.write(csv_content)
        return temp_file

    def _generate_csv_content(self, params):
        """Generate CSV content with parameters in SCOPE's expected row-based format"""
        sections = [
            ('PROSPECT', ['Cab', 'Cca', 'Cdm', 'Cw', 'Cs', 'Cant', 'Cp', 'Cbc', 'N', 'rho_thermal', 'tau_thermal']),
            ('Leaf_Biochemical', ['Vcmax25', 'BallBerrySlope', 'BallBerry0', 'Type', 'kV', 'Rdparam', 'Kn0', 'Knalpha', 'Knbeta']),
            ('Leaf_Biochemical_magnani', ['Tyear', 'beta', 'kNPQs', 'qLs', 'stressfactor']),
            ('Fluorescence', ['fqe']),
            ('Soil', ['spectrum', 'rss', 'rs_thermal', 'cs', 'rhos', 'lambdas', 'SMC', 'BSMBrightness', 'BSMlat', 'BSMlon']),
            ('Canopy', ['LAI', 'hc', 'LIDFa', 'LIDFb', 'leafwidth', 'Cv', 'crowndiameter']),
            ('Meteo', ['z', 'Rin', 'Ta', 'Rli', 'p', 'ea', 'u', 'Ca', 'Oa']),
            ('Aerodynamic', ['zo', 'd', 'Cd', 'rb', 'CR', 'CD1', 'Psicor', 'CSSOIL', 'rbs', 'rwc']),
            ('timeseries', ['startDOY', 'endDOY', 'LAT', 'LON', 'timezn']),
            ('Angles', ['tts', 'tto', 'psi'])
        ]

        # Determine number of simulations
        max_runs = 1
        for param in params.values():
            if isinstance(param, (list, tuple)):
                max_runs = max(max_runs, len(param))

        lines = []
        for section_name, param_names in sections:
            lines.append(f"{section_name},")
            for param in param_names:
                val = params.get(param, self.default_params.get(param, 0.0))
                # Convert scalar to single-element list
                if not isinstance(val, (list, tuple)):
                    val = [val]
                line = f"{param}," + ",".join(map(str, val))
                lines.append(line)
            lines.append(",")
        return "\r\n".join(lines)

    def _find_latest_output(self):
        """Find the newest output directory."""
        output_parent = self.scope_dir / "output"
        runs = sorted(output_parent.glob("example_run_*"), key=os.path.getmtime, reverse=True)
        if not runs:
            raise FileNotFoundError("No SCOPE output directories found.")
        return runs[0]

    def _read_data(self, filepath):
        """Reads numerical data from CSV, TXT, or BIN files."""
        if filepath.suffix.lower() == '.csv':
            try:
                # Check if the file is a spectral output (space-separated)
                is_spectral = (
                    'spectrum' in filepath.name or
                    'Esun' in filepath.name or
                    'Esky' in filepath.name or
                    'Eout' in filepath.name or
                    'fluorescence' in filepath.name
                )
                
                if is_spectral:
                    df = pd.read_csv(filepath, sep='\s+', header=0)
                else:
                    df = pd.read_csv(filepath, sep=',', header=0)
                
                # Convert to numeric and drop empty rows
                df = df.apply(pd.to_numeric, errors='coerce').dropna(how='all')
                return df.values.tolist()
            except Exception as e:
                print(f"Error reading {filepath}: {e}")
                return None

        elif filepath.suffix.lower() == '.txt':
            try:
                with open(filepath, 'r') as f:
                    return [[float(value) for value in line.split()] for line in f]
            except Exception as e:
                print(f"Error reading TXT file {filepath}: {e}")
                return None

        elif filepath.suffix.lower() == '.bin':  # Add .bin handling
            try:
                with open(filepath, 'rb') as f:
                    # Read all floats from the binary file
                    float_size = 4  # Assuming 4-byte floats (float32)
                    buffer = f.read()
                    num_floats = len(buffer) // float_size
                    data = struct.unpack(f'{num_floats}f', buffer)
                    return [list(data)] # Return as list of lists (consistent with CSV/TXT)
            except Exception as e:
                print(f"Error reading BIN file {filepath}: {e}")
                return None

        else:
            print(f"Unsupported file type: {filepath.suffix}")
            return None
            
    def _read_parameters(self, filepath):
        """Reads parameters from a CSV file, handling section headers."""
        params = {}
        with open(filepath, 'r') as f:
            reader = csv.reader(f)
            current_section = None
            for row in reader:
                if not row:
                    continue
                if row[0].endswith(','):
                    current_section = row[0].strip(',')
                elif len(row) >= 2 and current_section:
                    param_name = row[0].strip()
                    param_value = row[1].strip()
                    try:
                        param_value = float(param_value)
                    except ValueError:
                        pass
                    params[f"{current_section}.{param_name}"] = param_value
        return params

    def _read_setoptions(self, filepath):
      """Reads setoptions from a CSV file."""
      setoptions = {}
      with open(filepath, 'r') as f:
          reader = csv.reader(f)
          for row in reader:
              if row and len(row) >= 2:  # Check for non-empty and sufficient length
                  key = row[0].strip()
                  value = row[1].strip()
                  try:
                        value = int(value)  # Try converting to integer
                  except ValueError:
                        pass
                  setoptions[key] = value
      return setoptions


    def generate_results_json(self, output_dir: Path) -> Path:
        results = {'scalar_outputs': {}}
        


        # Input Parameters (from the backup file)
        if hasattr(self, 'temp_backup') and self.temp_backup.exists():
            results['input_parameters'] = self._read_parameters(self.temp_backup)

        setoptions_path = self.scope_dir / "input" / "setoptions.csv"
        if setoptions_path.exists():
            results['setoptions'] = self._read_setoptions(setoptions_path)
        # Check aPAR.csv for simulation count
        apar_file = output_dir / "aPAR.csv"
        if apar_file.exists():
            apar_data = self._read_data(apar_file)
            results['num_simulations'] = len(apar_data) if apar_data else 0
        else:
            results['num_simulations'] = 0

        # Selected scalar outputs (now including .bin files)
        results['scalar_outputs'] = {}
        scalar_files = {
            "aPAR": "aPAR.csv",       # Still check for CSV first
            "Eout": "Eout_spectrum.csv", #This too
            "Lo": "Lo_spectrum.csv", #This too
            "Esun": "Esun.csv", #This too
            "Esky": "Esky.csv", #This too
            "fluxes": "fluxes.csv",    # Add other CSV files as needed
            "rad": "radiation.csv",
            "fluorescence": "fluorescence.csv",
            "fluorescence_scalars": "fluorescence_scalars.csv",
        }
        for key, filename in scalar_files.items():
            filepath = output_dir / filename
            if filepath.exists():
                data = self._read_data(filepath)
                if data is not None:
                    results['scalar_outputs'][key] = data

        # Add handling for binary (.bin) files if saveCSV = 0
        if self.setoptions.get('saveCSV', 1) == 0: # Default to 1 if not present.
            bin_files = {
                "Rin": "Rin.bin",
                "Rli": "Rli.bin",
                "fluorescence": "fluorescence.bin",
                # Add other .bin files you want to include here
            }
            for key, filename in bin_files.items():
                filepath = output_dir / filename
                if filepath.exists():
                    data = self._read_data(filepath)
                    if data is not None:
                        results['scalar_outputs'][key] = data


        json_path = output_dir / "results.json"
        with open(json_path, 'w') as f:
            json.dump(results, f, indent=2)

        return json_path

    def _write_scope_csv(self, params, setoptions=None):
        """Write parameters directly to SCOPE's input_data.csv"""
        input_path = self.scope_dir / "input" / "input_data.csv"

        # Generate CSV content
        csv_content = self._generate_csv_content(params, setoptions)

        # Write directly to SCOPE's expected input location
        with open(input_path, 'w', newline='') as f:
            f.write(csv_content)

    def run(self, params, setoptions=None):
        """Run SCOPE, return results.json path.  Includes debugging prints."""
        
        try:
            if setoptions:
                self._update_setoptions(setoptions)  # Update setoptions.csv
            # Debugging: Print parameters before CSV generation
            print("=== Parameters before CSV generation ===")
            print(json.dumps(params, indent=2))

            # Prepare and write input data
            csv_content = self._generate_csv_content(params)
            print("\n=== Generated CSV Content ===")
            print(csv_content)  # Print the generated CSV content

            input_file = self._prepare_input(csv_content)


            # Run SCOPE
            self.eng.run_scope_wrapper_json(nargout=0)

            # Find output and generate results.json
            output_dir = self._find_latest_output()
            json_path = self.generate_results_json(output_dir)

            return json_path

        except matlab.engine.MatlabExecutionError as e:
            print(f"MATLAB Error: {str(e)}")
            raise
        except FileNotFoundError as e:
            print(f"File Not Found Error: {str(e)}")
            raise
        except Exception as e:
            print(f"An unexpected error occurred: {str(e)}")
            raise
    
    def _update_setoptions(self, user_options):
        """Enforce SCOPE.m's expected order for setoptions.csv (value,key)"""
        required_order = [
            (1, 'lite'),
            (1, 'calc_fluor'),    # Assuming fluorescence is needed
            (0, 'calc_planck'),
            (0, 'calc_xanthophyllabs'),
            (1, 'soilspectrum'),
            (0, 'Fluorescence_model'),
            (0, 'applTcorr'),
            (0, 'verify'),
            (1, 'saveCSV'),
            (0, 'mSCOPE'),
            (2, 'simulation'),    # Key line: Set to 2 for Lookup-Table mode
            (0, 'calc_directional'),
            (0, 'calc_vert_profiles'),
            (0, 'soil_heat_method'),
            (0, 'calc_rss_rbs'),
            (0, 'MoninObukhov'),
            (0, 'save_spectral')
        ]
        
        # Write to CSV in SCOPE's required sequence
        setoptions_path = self.scope_dir / "input" / "setoptions.csv"
        with open(setoptions_path, 'w', newline='') as f:
            writer = csv.writer(f)
            for value, key in required_order:
                writer.writerow([str(value), key])  # Value,Key as per your setup




def parse_fluorescence(output_dir):
    """Reads fluorescence data with wavelength alignment"""
    fluo_file = output_dir / "fluorescence.csv"
    wl_file = output_dir / "wlS.txt"
    
    if not fluo_file.exists() or not wl_file.exists():
        return None

    # Read wavelengths
    with open(wl_file, 'r') as f:
        wavelengths = [float(line.strip()) for line in f]
    
    # Read fluorescence data, skip header
    df = pd.read_csv(fluo_file, sep=',', header=0)
    
    # Transpose to [simulation][wavelength]
    return {
        'wavelengths': wavelengths,
        'spectra': df.values.T.tolist()
    }
    
# Example Usage (same as before, but now with correct multi-run handling)
if __name__ == "__main__":
    #Diagnostic output
    print("\n=== Path Diagnostics ===")
    print(f"Current working directory: {Path.cwd()}")
    print(f"Checking for 'SCOPE' in: {Path.cwd()}")

    try:
        with SCOPEWrapperMultiRun() as scope:
            print("\n=== Running Multi-Simulation Test ===")
            # Example parameters (can be adjusted)

            # Run a simulation with multiple LAI and Cab values
            multi_params = {
                'Cab': [30.0, 40.0, 50.0],  # 3 values
                'LAI': [2.0, 3.0],          # 2 values
                'hc': 2.0,                   # Fixed value
            }

            my_setoptions = {
                "simulation": 2,          # Enable Lookup-Table mode
                "calc_fluor": 1,          # Calculate fluorescence
                "soil_heat_method": 2,    # Use soil net radiation fraction
                "saveCSV": 1              # Output CSV files
            }

            results_path = scope.run(multi_params, my_setoptions)
            fluo_data = parse_fluorescence(output_dir)

            print("Success!")
            # You can now load and inspect the results.json here if needed
            with open(results_path, 'r') as f:
                results = json.load(f)

            if results.get('num_simulations', 0) > 0:
                print(f"Success: {results['num_simulations']} simulations")
                if 'aPAR' in results['scalar_outputs']:
                    print("First aPAR values:", results['scalar_outputs']['aPAR'][0])
            else:
                print("Error: No simulations detected. Check SCOPE logs.")
                
            print("\n=== Results ===")
            #print(json.dumps(results, indent=2)) # Print all results

            #Check number of simulations
            print(f"Number of simulations in output: {results.get('num_simulations')}") #Must be 3

            #Example: Print the first element of aPAR (if it exists)
            if 'scalar_outputs' in results and 'aPAR' in results['scalar_outputs']:
                print("\n=== Example Output (First element of aPAR) ===")
                print(results['scalar_outputs']['aPAR'][0]) # First simulation
                #print(results['scalar_outputs']['aPAR'])  # Print all
            else:
                print("aPAR data not found in output.")



    except Exception as e:
        print(f"Error during SCOPE execution: {e}")


=== Path Diagnostics ===
Current working directory: /Users/mirkomorello/Documents/Università/MSc_Sensors_Imaging/Final_Project
Checking for 'SCOPE' in: /Users/mirkomorello/Documents/Università/MSc_Sensors_Imaging/Final_Project

=== Running Multi-Simulation Test ===
=== Parameters before CSV generation ===
{
  "Cab": [
    30.0,
    40.0,
    50.0
  ],
  "LAI": [
    2.0,
    3.0
  ],
  "hc": 2.0
}

=== Generated CSV Content ===
PROSPECT,
Cab,30.0,40.0,50.0
Cca,10.0
Cdm,0.012
Cw,0.009
Cs,0.0
Cant,1.0
Cp,0.0
Cbc,0.0
N,1.5
rho_thermal,0.01
tau_thermal,0.01
,
Leaf_Biochemical,
Vcmax25,60.0
BallBerrySlope,8.0
BallBerry0,0.01
Type,0.0
kV,0.64
Rdparam,0.015
Kn0,2.48
Knalpha,2.83
Knbeta,0.114
,
Leaf_Biochemical_magnani,
Tyear,15.0
beta,0.51
kNPQs,0.0
qLs,1.0
stressfactor,1.0
,
Fluorescence,
fqe,0.01
,
Soil,
spectrum,1.0
rss,500.0
rs_thermal,0.06
cs,1180.0
rhos,1800.0
lambdas,1.55
SMC,25.0
BSMBrightness,0.5
BSMlat,25.0
BSMlon,45.0
,
Canopy,
LAI,2.0,3.0
hc,2.0
LIDFa,-0.35
LIDFb,-0.15
leafwidt

In [93]:
def parse_fluorescence(output_dir):
    """Find and read fluorescence output files directly from SCOPE's output folder"""
    results = {}
    
    # 1. Find all fluorescence-related files
    fluo_files = {
        'scalars': 'fluorescence_scalars.csv',  # Integrated fluorescence values
        'spectra': 'fluorescence.csv',          # Full fluorescence spectra
        'hemispheric': 'fluorescence_hemis.csv' # Hemispheric fluorescence
    }
    
    # 2. Read each file
    for key, fname in fluo_files.items():
        filepath = Path(output_dir) / fname
        if filepath.exists():
            df = pd.read_csv(filepath, header=None)
            results[key] = df.values.tolist()
            
    # 3. Get wavelengths (from first simulation only)
    wavelength_file = Path(output_dir) / 'wlS.txt'
    if wavelength_file.exists():
        with open(wavelength_file, 'r') as f:
            wavelengths = [float(line.strip()) for line in f]
        results['wavelengths'] = wavelengths
            
    return results

# Usage:
with SCOPEWrapper() as scope:
    output_dir = scope.run(multi_params)
    fluo_data = parse_fluorescence(output_dir)
    
# Convert to pandas DataFrame
df_fluorescence = pd.DataFrame({
    'wavelength': fluo_data['wavelengths'],
    'F_red': fluo_data['spectra'][0],     # First simulation
    'F_far_red': fluo_data['spectra'][1]  # Second simulation
})
print(df_fluorescence.head())

=== Parameters before CSV generation ===
{
  "Cab": [
    30.0,
    40.0,
    50.0
  ],
  "Cca": [
    10.0,
    10.0,
    10.0
  ],
  "Cdm": [
    0.012,
    0.012,
    0.012
  ],
  "Cw": [
    0.009,
    0.009,
    0.009
  ],
  "Cs": [
    0.0,
    0.0,
    0.0
  ],
  "N": [
    1.5,
    1.5,
    1.5
  ],
  "LAI": [
    2.0,
    3.0,
    4.0
  ],
  "hc": [
    2.0,
    2.0,
    2.0
  ],
  "LIDFa": [
    -0.35,
    -0.35,
    -0.35
  ],
  "LIDFb": [
    -0.15,
    -0.15,
    -0.15
  ],
  "fqe": [
    0.01,
    0.01,
    0.01
  ],
  "Rin": [
    850.0,
    850.0,
    850.0
  ],
  "Rli": [
    300.0,
    300.0,
    300.0
  ],
  "Ta": [
    20.0,
    20.0,
    20.0
  ],
  "p": [
    970.0,
    970.0,
    970.0
  ],
  "tts": [
    30.0,
    30.0,
    30.0
  ],
  "tto": [
    0.0,
    0.0,
    0.0
  ],
  "psi": [
    0.0,
    0.0,
    0.0
  ],
  "rss": [
    500.0,
    500.0,
    500.0
  ],
  "SMC": [
    25.0,
    25.0,
    25.0
  ]
}

=== Generated CSV Content ===
PROSPECT,
Cab,Cca,Cdm,C

Error using run_scope_wrapper_json
SCOPE_ERROR: Unrecognized function or variable 'n_col'.

Error in SCOPE (line 469)
    bin_to_csv(fnames, V, vmax, n_col, telmax)

Error in run_scope_wrapper_json (line 8)
        SCOPE; 



MatlabExecutionError: 
  File /Users/mirkomorello/Documents/Università/MSc_Sensors_Imaging/Final_Project/SCOPE/run_scope_wrapper_json.m, line 11, in run_scope_wrapper_json
SCOPE_ERROR: Unrecognized function or variable 'n_col'.

Error in SCOPE (line 469)
    bin_to_csv(fnames, V, vmax, n_col, telmax)

Error in run_scope_wrapper_json (line 8)
        SCOPE; 


In [109]:
class SCOPEHandler:
    """Improved SCOPE integration handler with MATLAB engine"""
    
    def __init__(self, scope_path):
        self.scope_path = Path(scope_path).resolve()
        self.eng = None
        self._validate_paths()

    def _validate_paths(self):
        required_files = {
            'main': self.scope_path / 'SCOPE.m',
            'wrapper': self.scope_path / 'run_scope_wrapper_json.m',
            'input_dir': self.scope_path / 'input'
        }
        
        missing = [k for k,v in required_files.items() if not v.exists()]
        if missing:
            raise FileNotFoundError(
                f"Missing required SCOPE files: {', '.join(missing)}\n"
                f"Verify SCOPE installation at: {self.scope_path}"
            )

    def __enter__(self):
        self.eng = matlab.engine.start_matlab('-nodisplay')
        self.eng.addpath(str(self.scope_path), nargout=0)
        self.eng.cd(str(self.scope_path), nargout=0)
        return self

    def __exit__(self, *exc):
        self.eng.quit()
        return False

    def run_simulation(self, parameters):
        """Execute SCOPE with parameter dictionary"""
        with NamedTemporaryFile(mode='w', suffix='.json', delete=False) as f:
            json.dump(parameters, f)
            temp_json = f.name
        
        try:
            output = self.eng.run_scope_wrapper_json(
                temp_json, 
                str(self.scope_path / 'output' / 'results.json'),
                nargout=1
            )
            return self._parse_output(output)
        finally:
            os.unlink(temp_json)
            shutil.rmtree(self.scope_path / 'output', ignore_errors=True)

    def _parse_output(self, output):
        """Parse MATLAB output structure to Python dict"""
        return {
            'wavelength': np.array(output['wavelength']).ravel(),
            'reflectance': np.array(output['reflectance']).ravel(),
            'fluorescence': np.array(output['fluorescence']).ravel()
        }

In [9]:
import subprocess
from tempfile import NamedTemporaryFile


class MODTRANHandler:
    """MODTRAN simulation handler"""
    
    def __init__(self, modtran_path, template_file):
        self.modtran_path = Path(modtran_path).resolve()
        self.template = self.modtran_path / template_file
        self._validate_setup()

    def _validate_setup(self):
        if not (self.modtran_path / 'bin' / 'Mod5_mac.exe').exists():
            raise FileNotFoundError("MODTRAN executable not found")

    def run_simulation(self, params, case_id):
        """Execute MODTRAN simulation with parameters"""
        tp5_content = self._generate_tp5(params)
        
        with NamedTemporaryFile(mode='w', suffix='.tp5', delete=False) as f:
            f.write(tp5_content)
            tp5_path = Path(f.name)
        
        try:
            subprocess.run(
                [str(self.modtran_path / 'bin' / 'mod5_win64.exe'), str(tp5_path)],
                cwd=self.modtran_path,
                check=True,
                capture_output=True
            )
            return self._parse_output(tp5_path)
        finally:
            tp5_path.unlink()

    def _generate_tp5(self, params):
        """Generate TP5 file from template and parameters"""
        with open(self.template, 'r') as f:
            content = f.read()
        
        replacements = {
            '{H1}': f"{params['sensor_altitude']:.3f}",
            '{SZA}': f"{params['sza']:.1f}",
            '{VIS}': f"{params['visibility']:.2f}",
            # Add more parameter replacements as needed
        }
        
        for ph, val in replacements.items():
            content = content.replace(ph, val)
        
        return content

    def _parse_output(self, tp5_path):
        """Parse MODTRAN output files (.tp7)"""
        tp7_path = tp5_path.with_suffix('.tp7')
        
        # Implement actual parsing based on your MODTRAN output format
        return {
            'direct_transmittance': np.random.rand(100),
            'diffuse_transmittance': np.random.rand(100),
            'spherical_albedo': np.random.rand(100),
            'fluorescence_transmittance': np.random.rand(100)
        }


In [6]:
class SyntheticDataGenerator:
    """Main synthetic dataset generation pipeline"""
    
    def __init__(self, config):
        self.config = config
        self.scope = SCOPEHandler(config['scope_path'])
        self.modtran = MODTRANHandler(
            config['modtran_path'],
            config['modtran_template']
        )
        self.output_dir = Path(config['output_path']).resolve()
        self.output_dir.mkdir(parents=True, exist_ok=True)

    def generate_dataset(self, num_samples=1000):
        """Generate complete synthetic dataset"""
        param_ranges = {
            'lai': (0.5, 6.0),
            'cab': (20, 80),
            'sza': (20, 70),
            'visibility': (5, 50),
            'sensor_altitude': (350, 1500)
        }

        for i in range(num_samples):
            params = self._random_parameters(param_ranges)
            try:
                # Run SCOPE simulation
                with self.scope as scope:
                    scope_result = scope.run_simulation(params)
                
                # Run MODTRAN simulation
                modtran_result = self.modtran.run_simulation(params, i)
                
                # Calculate final radiance
                radiance = self._calculate_radiance(
                    scope_result['reflectance'],
                    scope_result['fluorescence'],
                    modtran_result
                )
                
                # Save sample
                self._save_sample(i, radiance, params)

            except Exception as e:
                print(f"Error generating sample {i}: {str(e)}")
                continue

    def _random_parameters(self, ranges):
        """Generate random parameters within specified ranges"""
        return {
            'lai': np.random.uniform(*ranges['lai']),
            'cab': np.random.uniform(*ranges['cab']),
            'sza': np.random.uniform(*ranges['sza']),
            'visibility': np.random.uniform(*ranges['visibility']),
            'sensor_altitude': np.random.uniform(*ranges['sensor_altitude'])
        }

    def _calculate_radiance(self, reflectance, fluorescence, modtran):
        """Calculate at-sensor radiance using physics model"""
        return (
            (modtran['direct_irradiance'] * modtran['direct_transmittance'] +
            modtran['diffuse_irradiance'] * modtran['diffuse_transmittance'])
            * reflectance / (1 - reflectance * modtran['spherical_albedo'])
        ) + fluorescence * modtran['fluorescence_transmittance']

    def _save_sample(self, idx, radiance, params):
        """Save sample as compressed numpy file"""
        np.savez_compressed(
            self.output_dir / f'sample_{idx:04d}.npz',
            radiance=np.tile(radiance, (17, 17, 1)),  # Create 17x17 patch
            parameters=params,
            wavelength=self.scope.wavelength
        )

In [7]:
if __name__ == "__main__":
    config = {
        'scope_path': '/path/to/SCOPE',
        'modtran_path': '/path/to/MODTRAN',
        'modtran_template': 'HyPlant-FLUO_Modtran5_base_v1.tp5',
        'output_path': './synthetic_dataset'
    }

    generator = SyntheticDataGenerator(config)
    
    print("Starting synthetic data generation...")
    generator.generate_dataset(num_samples=100)
    print("Dataset generation completed successfully!")

FileNotFoundError: Missing required SCOPE files: main, wrapper, input_dir
Verify SCOPE installation at: /path/to/SCOPE

In [ ]:
import numpy as np
import scipy.interpolate
from scipy.ndimage import gaussian_filter
# import subprocess # No longer used directly
# import os # No longer needed

from pathlib import Path
import pandas as pd  # For reading MODTRAN output
import matlab.engine  # For interacting with MATLAB


# --- 1. Define Parameter Ranges ---
# *SAME AS BEFORE*, but adjusted for your specific simulation needs

# SCOPE Parameters (Example)
lai_values = np.linspace(0.5, 6, 20)
cab_values = np.linspace(20, 80, 20)
cbrown_values = [0]  # No variation
cw_values = np.linspace(0.008, 0.03, 20)
cm_values = np.linspace(0.004, 0.01, 20)
lidfa_values = [-0.4]  # Spherical leaf angle distribution
lidfb_values = [0.0]  # Spherical leaf angle distribution
tts_values = np.linspace(20, 70, 6)  # Vary solar zenith angle
tto_values = [0]  # Nadir viewing
psi_values = [0]   # We don't need it

# MODTRAN Parameters (Example)
model = 2  # Mid-Latitude Summer
i_aero = 2  # Rural aerosol model
vis_values = np.linspace(5, 50, 10)  # Vary visibility!
gndalt = 0.6 # km.

# HyPlant parameters (same as before)
hyplant_fwhm = 0.27
hyplant_ssi = 0.11
hyplant_wavelengths = np.arange(670, 780, hyplant_ssi)
wavelengths = np.arange(750, 770.001, 0.005) #high-resolution for MODTRAN and SCOPE

# --- 2. Paths (CRUCIAL: Adjust these to your system) ---

modtran_exe_path = Path("./Final_Project/MODTRAN5/bin/Mod5_win64.exe")
modtran_data_path = Path("./Final_Project/MODTRAN5/bin/DATA")  # Path to DATA folder
scope_path = Path("./Final_Project/SCOPE") #Path to the SCOPE folder.
matlab_functions_path = Path('./Final_Project/MODTRAN5/matlab-modtran-5-aba70d781805') # Path to the matlab functions provided.
matlab_t14_path = Path('./Final_Project/MODTRAN5/modtran5_acd_t14') # Path to soltir_tp7.m

# --- 3. Helper Functions (Modified) ---

# Modified to use matlab.engine.
def run_modtran(params_matlab, matlab_engine):
    """
    Runs MODTRAN using the provided parameters.

    Args:
        params_matlab (matlab.engine.struct): A MATLAB structure containing the
            MODTRAN input parameters.
        matlab_engine: The MATLAB engine instance

    Returns:
        tuple: A tuple containing the interpolated solar irradiance and
               atmospheric transfer functions (t1-t12).
    """
    try:
        # Call the MATLAB function run_MODTRAN5_AC_PAR_4RUNS.
        # Make sure the path is correct.
        # nargout=4 is VERY IMPORTANT: it tells MATLAB to return 4 values.
        result = matlab_engine.run_MODTRAN5_AC_PAR_4RUNS(
            './', 'MODTRAN_SCOPE_ATM', params_matlab, 'FLUO', 'A', 1, 0, 0, nargout=4
        )

        # Extract results and convert to NumPy arrays.
        # The .flatten() converts 2D arrays to 1D.
        wvlLUT          = np.array(result[0]).flatten()
        T14            = np.array(result[1])
        DV             = np.array(result[2]).flatten()
        # atmospheric_ch  = np.array(result[3]).flatten() #Not needed.

        # Extract the relevant transfer functions (t1-t12).
        #  Make SURE you're extracting the right columns.
        t1 = T14[:, 0]
        t2 = T14[:, 1]
        t3 = T14[:, 2]
        t4 = T14[:, 3]
        t5 = T14[:, 4]
        t6 = T14[:, 5]
        t7 = T14[:, 6]
        t8 = T14[:, 7]
        t9 = T14[:, 8]
        t10 = T14[:, 9]
        t11 = T14[:, 10]
        # t12 is all zeros
        t12 = np.zeros_like(t1)
        
        # Extract solar irradiance
        solar_irr = T14[:,14] # The solar irradiance should be here
        
        # Interpolate to high-res wavelengths
        t_values_interp = {}  # Store interpolated t values in a dictionary

        f_interp_solar_irr = scipy.interpolate.interp1d(wvlLUT, solar_irr, bounds_error = False, fill_value = 'extrapolate')
        solar_irradiance_interp = f_interp_solar_irr(wavelengths)
        
        for i in range(1, 13):  # t1 to t12
            if i != 12: # Don't interpolate t12
                t_func = scipy.interpolate.interp1d(wvlLUT, T14[:, i-1], bounds_error=False, fill_value='extrapolate') # Extrapolate if needed
                t_values_interp[f't{i}'] = t_func(wavelengths) #interpolate
            else:
               t_values_interp['t12'] = t12


    except matlab.engine.MatlabExecutionError as e:
        print(f"MATLAB Error: {e}")
        return None, None # Return None if MODTRAN fails.

    return solar_irradiance_interp, t_values_interp



def calculate_t_values(solar_irradiance, transmittance_dir, transmittance_dif_down, transmittance_dif_up, path_radiance):
    # THIS FUNCTION IS NO LONGER NEEDED BECAUSE WE DIRECTLY OBTAIN THE t
    # VALUES FROM THE MODTRAN OUTPUT.
    pass

def simulate_hyplant(lat_s, wavelengths, hyplant_wavelengths, hyplant_fwhm, delta_lambda=0, delta_sigma=0):
    # ... (Same as before)
    wavelengths_shifted = wavelengths + delta_lambda
    fwhm_shifted = hyplant_fwhm + delta_sigma
    sigma = fwhm_shifted / (2 * np.sqrt(2 * np.log(2)))
    lat_s_convolved = gaussian_filter(lat_s, sigma / (wavelengths[1] - wavelengths[0]))
    f_interp = scipy.interpolate.interp1d(wavelengths_shifted, lat_s_convolved, bounds_error=False, fill_value=0)
    lhyp = f_interp(hyplant_wavelengths)
    return lhyp



def run_scope(params, eng):
    '''
    Runs the SCOPE model given the parameters.
    
    Args:
    params: A dictionary with the parameters for the model.
    eng: The MATLAB engine instance.
    
    Returns:
    reflectance, fluorescence: The reflectance and fluorescence simulated by SCOPE.
    
    '''
    try:

        # Call SCOPE (replace with your actual SCOPE call)
        # result = eng.run_scope_wrapper(params, nargout=2)  # Adjust nargout
        result = eng.SCOPE(params, nargout=1)  # Call the main SCOPE.m function, and get only one structure output.
        # I assume here that the result is a MATLAB struct
        # containing the fields 'reflectance' and 'fluorescence'.
        # *ADJUST* this if your SCOPE setup returns the data differently.
        # The following two lines *MIGHT* need to be transposed; test this.
        reflectance  = np.array(result['reflectance']).flatten()  # Extract and convert to NumPy array.
        fluorescence = np.array(result['Fluspect']).flatten()    # Extract and convert to NumPy array.
    except matlab.engine.MatlabExecutionError as e:
        print(f"MATLAB Error: {e}")
        # Handle the error appropriately (e.g., log it, skip this simulation)
        reflectance = None  # Or some other default value
        fluorescence = None

    return reflectance, fluorescence
# --- 4. Main Simulation Loop ---
# Start MATLAB engine outside the loop
eng = matlab.engine.start_matlab()

# Add paths to the MATLAB engine
eng.addpath(str(matlab_functions_path), nargout=0)
eng.addpath(str(matlab_t14_path), nargout=0)
eng.addpath(str(scope_path), nargout=0)

# Pre-allocate for results
num_simulations = (len(lai_values) * len(cab_values) * len(cbrown_values) * len(cw_values) *
                    len(cm_values) * len(tts_values) * len(vis_values) * len(lidfa_values) * len(lidfb_values))

num_pixels = 17 * 17
lhyp_data = np.zeros((num_simulations, num_pixels, len(hyplant_wavelengths)))
parameter_data = np.zeros((num_simulations, 9))  # Adjust for all your parameters


simulation_index = 0

for lai in lai_values:
    for cab in cab_values:
        for cbrown in cbrown_values:
            for cw in cw_values:
                for cm in cm_values:
                    for tts in tts_values:
                        for lidfa in lidfa_values:
                            for lidfb in lidfb_values:
                                for vis in vis_values:

                                    # --- Prepare MODTRAN input parameters (as a MATLAB struct) ---
                                    params_modtran = eng.struct()
                                    params_modtran['ATM'] = eng.struct()  # Nested structure
                                    params_modtran['ATM']['LSUNFL'] = 1.0  # Use a specific solar irradiance file
                                    params_modtran['ATM']['MODEL'] = float(model)
                                    params_modtran['ATM']['IHAZE'] = float(i_aero)
                                    params_modtran['ATM']['VIS'] = float(vis)
                                    params_modtran['ATM']['GNDALT'] = float(gndalt)
                                    params_modtran['ATM']['H1'] = 100.0 #Fixed
                                    params_modtran['ATM']['SZA'] = float(tts) # Solar Zenith angle
                                    params_modtran['ATM']['WVL'] = matlab.double([float(wavelengths.min()), float(wavelengths.max())])  # Convert to MATLAB double array

                                    # --- Run MODTRAN ---
                                    solar_irr, t_values = run_modtran(params_modtran, eng)

                                    if solar_irr is None or t_values is None:
                                        print(f"Skipping simulation {simulation_index} due to MODTRAN error.")
                                        simulation_index += 1 # Increment to avoid holes in output arrays.
                                        continue  # Skip to the next iteration


                                    # --- Prepare SCOPE input parameters ---
                                    # Example: You may need to adjust based on your SCOPE setup.
                                    params_scope = eng.struct() # Create the struct.
                                    params_scope['LAI']    = matlab.double([lai])
                                    params_scope['Cab']    = matlab.double([cab])
                                    params_scope['Cbrown'] = matlab.double([cbrown])
                                    params_scope['Cw']     = matlab.double([cw])
                                    params_scope['Cm']     = matlab.double([cm])
                                    params_scope['tts']    = matlab.double([tts])
                                    params_scope['tto']    = matlab.double([tto_values[0]]) # tto and psi are fixed.
                                    params_scope['psi']    = matlab.double([psi_values[0]])
                                    params_scope['lidfa']  = matlab.double([lidfa])
                                    params_scope['lidfb']  = matlab.double([lidfb])

                                    # --- Run SCOPE ---
                                    reflectance, fluorescence = run_scope(params_scope, eng)
                                    if reflectance is None or fluorescence is None:
                                        print(f"Skipping simulation {simulation_index} due to SCOPE error.")
                                        simulation_index += 1
                                        continue

                                    # Interpolate SCOPE output to MODTRAN wavelengths
                                    f_interp_R = scipy.interpolate.interp1d(np.arange(400, 2501), reflectance, bounds_error=False, fill_value='extrapolate')
                                    reflectance_interp = f_interp_R(wavelengths)

                                    f_interp_F = scipy.interpolate.interp1d(np.arange(400, 2501), fluorescence, bounds_error=False, fill_value='extrapolate')
                                    fluorescence_interp = f_interp_F(wavelengths)


                                    # --- Calculate At-Sensor Radiance (Lat-s) ---
                                    r_s = reflectance_interp  # Reflectance of the target
                                    f_s = fluorescence_interp  # Fluorescence of the target
                                    r_surr = r_s  # Homogeneous assumption
                                    f_surr = f_s  # Homogeneous assumption
                                    e_0 = solar_irr * np.cos(np.deg2rad(tts))  # Modulate by solar zenith angle

                                    l_r = e_0 * (t_values['t1'] + t_values['t7'] * r_s + t_values['t8'] * r_surr + t_values['t10'] * r_surr + t_values['t9'] * r_s) / (1 - t_values['t2'] * r_surr)
                                    l_f = t_values['t5'] * (f_s + (t_values['t6'] + t_values['t11'] * r_surr) * f_surr) / (1-t_values['t2'] * r_surr) # Bayat (2020)
                                    lat_s = l_r / np.pi + l_f

                                    # --- Simulate HyPlant Observation ---
                                    lhyp = simulate_hyplant(lat_s, wavelengths, hyplant_wavelengths, hyplant_fwhm)
                                    # Create a 17x17 patch (homogeneous in this example)
                                    lhyp_patch = np.tile(lhyp, (num_pixels, 1))

                                    # --- Store Results ---
                                    lhyp_data[simulation_index, :, :] = lhyp_patch
                                    parameter_data[simulation_index, :] = [lai, cab, cbrown, cw, cm, lidfa, lidfb, tts, vis]

                                    simulation_index += 1

# --- 5. Save Data ---
np.savez('synthetic_dataset.npz', lhyp=lhyp_data, parameters=parameter_data)
# Close MATLAB engine when finished
eng.quit()

print("Simulations Complete!")

> In path (line 109)
In addpath>doPathAddition (line 116)
In addpath (line 80)
> In path (line 109)
In addpath>doPathAddition (line 116)
In addpath (line 80)
> In path (line 109)
In addpath>doPathAddition (line 116)
In addpath (line 80)


Unrecognized function or variable 'run_MODTRAN5_AC_PAR_4RUNS'.



MATLAB Error: Undefined function 'run_MODTRAN5_AC_PAR_4RUNS' for input arguments of type 'struct'.

Skipping simulation 0 due to MODTRAN error.


Unrecognized function or variable 'run_MODTRAN5_AC_PAR_4RUNS'.

Unrecognized function or variable 'run_MODTRAN5_AC_PAR_4RUNS'.

Unrecognized function or variable 'run_MODTRAN5_AC_PAR_4RUNS'.

Unrecognized function or variable 'run_MODTRAN5_AC_PAR_4RUNS'.

Unrecognized function or variable 'run_MODTRAN5_AC_PAR_4RUNS'.

Unrecognized function or variable 'run_MODTRAN5_AC_PAR_4RUNS'.

Unrecognized function or variable 'run_MODTRAN5_AC_PAR_4RUNS'.

Unrecognized function or variable 'run_MODTRAN5_AC_PAR_4RUNS'.

Unrecognized function or variable 'run_MODTRAN5_AC_PAR_4RUNS'.

Unrecognized function or variable 'run_MODTRAN5_AC_PAR_4RUNS'.

Unrecognized function or variable 'run_MODTRAN5_AC_PAR_4RUNS'.



MATLAB Error: Undefined function 'run_MODTRAN5_AC_PAR_4RUNS' for input arguments of type 'struct'.

Skipping simulation 1 due to MODTRAN error.
MATLAB Error: Undefined function 'run_MODTRAN5_AC_PAR_4RUNS' for input arguments of type 'struct'.

Skipping simulation 2 due to MODTRAN error.
MATLAB Error: Undefined function 'run_MODTRAN5_AC_PAR_4RUNS' for input arguments of type 'struct'.

Skipping simulation 3 due to MODTRAN error.
MATLAB Error: Undefined function 'run_MODTRAN5_AC_PAR_4RUNS' for input arguments of type 'struct'.

Skipping simulation 4 due to MODTRAN error.
MATLAB Error: Undefined function 'run_MODTRAN5_AC_PAR_4RUNS' for input arguments of type 'struct'.

Skipping simulation 5 due to MODTRAN error.
MATLAB Error: Undefined function 'run_MODTRAN5_AC_PAR_4RUNS' for input arguments of type 'struct'.

Skipping simulation 6 due to MODTRAN error.
MATLAB Error: Undefined function 'run_MODTRAN5_AC_PAR_4RUNS' for input arguments of type 'struct'.

Skipping simulation 7 due to MODTRAN

...

MATLAB Error: Undefined function 'run_MODTRAN5_AC_PAR_4RUNS' for input arguments of type 'struct'.

Skipping simulation 1007 due to MODTRAN error.
MATLAB Error: Undefined function 'run_MODTRAN5_AC_PAR_4RUNS' for input arguments of type 'struct'.

Skipping simulation 1008 due to MODTRAN error.
MATLAB Error: Undefined function 'run_MODTRAN5_AC_PAR_4RUNS' for input arguments of type 'struct'.

Skipping simulation 1009 due to MODTRAN error.
MATLAB Error: Undefined function 'run_MODTRAN5_AC_PAR_4RUNS' for input arguments of type 'struct'.

Skipping simulation 1010 due to MODTRAN error.
MATLAB Error: Undefined function 'run_MODTRAN5_AC_PAR_4RUNS' for input arguments of type 'struct'.

Skipping simulation 1011 due to MODTRAN error.
MATLAB Error: Undefined function 'run_MODTRAN5_AC_PAR_4RUNS' for input arguments of type 'struct'.

Skipping simulation 1012 due to MODTRAN error.
MATLAB Error: Undefined function 'run_MODTRAN5_AC_PAR_4RUNS' for input arguments of type 'struct'.

Skipping simulatio

Unrecognized function or variable 'run_MODTRAN5_AC_PAR_4RUNS'.

Unrecognized function or variable 'run_MODTRAN5_AC_PAR_4RUNS'.

Unrecognized function or variable 'run_MODTRAN5_AC_PAR_4RUNS'.

Unrecognized function or variable 'run_MODTRAN5_AC_PAR_4RUNS'.

Unrecognized function or variable 'run_MODTRAN5_AC_PAR_4RUNS'.

Unrecognized function or variable 'run_MODTRAN5_AC_PAR_4RUNS'.

Unrecognized function or variable 'run_MODTRAN5_AC_PAR_4RUNS'.

Unrecognized function or variable 'run_MODTRAN5_AC_PAR_4RUNS'.

Unrecognized function or variable 'run_MODTRAN5_AC_PAR_4RUNS'.

Unrecognized function or variable 'run_MODTRAN5_AC_PAR_4RUNS'.

Unrecognized function or variable 'run_MODTRAN5_AC_PAR_4RUNS'.

Unrecognized function or variable 'run_MODTRAN5_AC_PAR_4RUNS'.

Unrecognized function or variable 'run_MODTRAN5_AC_PAR_4RUNS'.

Unrecognized function or variable 'run_MODTRAN5_AC_PAR_4RUNS'.

Unrecognized function or variable 'run_MODTRAN5_AC_PAR_4RUNS'.

Unrecognized function or variable 'run_M

MATLAB Error: Undefined function 'run_MODTRAN5_AC_PAR_4RUNS' for input arguments of type 'struct'.

Skipping simulation 1026 due to MODTRAN error.
MATLAB Error: Undefined function 'run_MODTRAN5_AC_PAR_4RUNS' for input arguments of type 'struct'.

Skipping simulation 1027 due to MODTRAN error.
MATLAB Error: Undefined function 'run_MODTRAN5_AC_PAR_4RUNS' for input arguments of type 'struct'.

Skipping simulation 1028 due to MODTRAN error.
MATLAB Error: Undefined function 'run_MODTRAN5_AC_PAR_4RUNS' for input arguments of type 'struct'.

Skipping simulation 1029 due to MODTRAN error.
MATLAB Error: Undefined function 'run_MODTRAN5_AC_PAR_4RUNS' for input arguments of type 'struct'.

Skipping simulation 1030 due to MODTRAN error.
MATLAB Error: Undefined function 'run_MODTRAN5_AC_PAR_4RUNS' for input arguments of type 'struct'.

Skipping simulation 1031 due to MODTRAN error.
MATLAB Error: Undefined function 'run_MODTRAN5_AC_PAR_4RUNS' for input arguments of type 'struct'.

Skipping simulatio

Unrecognized function or variable 'run_MODTRAN5_AC_PAR_4RUNS'.

Unrecognized function or variable 'run_MODTRAN5_AC_PAR_4RUNS'.

Unrecognized function or variable 'run_MODTRAN5_AC_PAR_4RUNS'.

Unrecognized function or variable 'run_MODTRAN5_AC_PAR_4RUNS'.

Unrecognized function or variable 'run_MODTRAN5_AC_PAR_4RUNS'.

Unrecognized function or variable 'run_MODTRAN5_AC_PAR_4RUNS'.

Undefined function 'run_MODTRAN5_AC_PAR_4RUNS' for input arguments of type 'struct'.



TypeError: 'NoneType' object is not subscriptable